# Diogo Data Reproduction

In [27]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import tensorflow as tf
import random
import qiskit as qk

from tqdm.notebook import tqdm
from qiskit.quantum_info import Operator

from loss_functions import ProbabilityMSE, channel_fidelity_loss, ProbabilityRValue
from optimization import ModelQuantumMap
from kraus_channels import KrausMap
from quantum_channel import channel_fidelity
from quantum_tools import reshuffle, state_fidelity
from spam import IdealSPAM
from utils import saver

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

### Fit Quantum Channel

In [9]:
# load from file with numpy
data = np.load('data/data_diogo/Data.npy', allow_pickle=True)

In [10]:
probs = data[0][0]
targets = tf.convert_to_tensor([i for i in probs], dtype=tf.complex128)

tf.Tensor(
[[0.008 +0.j 0.2992+0.j 0.1414+0.j ... 0.2132+0.j 0.25  +0.j 0.025 +0.j]
 [0.0138+0.j 0.0274+0.j 0.0958+0.j ... 0.2204+0.j 0.2328+0.j 0.2844+0.j]
 [0.191 +0.j 0.0774+0.j 0.257 +0.j ... 0.1362+0.j 0.0404+0.j 0.0548+0.j]
 ...
 [0.071 +0.j 0.1154+0.j 0.0504+0.j ... 0.1544+0.j 0.0322+0.j 0.2344+0.j]
 [0.0648+0.j 0.0626+0.j 0.0174+0.j ... 0.4764+0.j 0.0246+0.j 0.0994+0.j]
 [0.0992+0.j 0.0292+0.j 0.0992+0.j ... 0.1898+0.j 0.1816+0.j 0.1794+0.j]], shape=(324, 8), dtype=complex128)


In [11]:
n = 3
N = 324


string = data[1][0]
input_list = []
output_list = []
for i in range(N):
    circuit = qk.QuantumCircuit(n)
    for j in range(n):

        if string[i][3*j] == 0: # X
            if string[i][3*j+1] == 0:
                circuit.ry(np.pi / 2, j)
            else:
                circuit.ry(-np.pi / 2, j)

        if string[i][3*j] == 1:  # Y
            if string[i][3*j+1] == 0:
                circuit.rx(-np.pi / 2, j)
            else:
                circuit.rx(np.pi / 2, j)
        
        if string[i][3*j] == 2: # Z
            if string[i][3*j+1] == 0:
                pass
            else:
                circuit.x(j)

    input = Operator(circuit.reverse_bits()).data

    circuit = qk.QuantumCircuit(n)
    for j in range(n):

        if string[i][3*j+2] == 0: # X
            circuit.ry(np.pi / 2, j)


        if string[i][3*j+2] == 1: #Y
            circuit.rx(-np.pi / 2, j)

        
        if string[i][3*j+2] == 2: # Z
            pass

    output = Operator(circuit.reverse_bits()).data

    input_list.append(input)
    output_list.append(output)

inputs = [tf.convert_to_tensor(input_list, dtype=tf.complex128), tf.convert_to_tensor(output_list, dtype=tf.complex128)]


In [6]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

d = 2**n

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                            rank = d**2,
                                            spam = IdealSPAM(d),
                                            ),
                        loss_function = ProbabilityMSE(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    )



NameError: name 'n' is not defined

In [14]:
model.train(inputs = inputs,
            targets = targets,
            num_iter = 2000,
            N = 500,
            verbose=True,
        )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.4411133161804305 None
0.08626773336284373 None
0.015268444545692143 None
0.00501614649378974 None
0.002623168580628299 None
0.0018115112109788965 None
0.0014647779733455984 None
0.0012919223174343237 None
0.001195518677941299 None
0.0011369366102843546 None
0.0010988768464046225 None
0.0010728337181615572 None
0.0010542908863367874 None
0.001040685269774022 None
0.001030474132957528 None
0.0010226794188840018 None
0.0010166526011747136 None
0.001011947118056746 None
0.0010082457864213467 None
0.0010053175643383312 None
0.0010030114277233676 None


In [23]:
choi = model.channel.choi

data2 = np.load('data/data_diogo/Tgt_Map.npy', allow_pickle=True)
choi_ref = reshuffle(data2[0])

In [19]:
channel_fid = state_fidelity(choi, choi_ref)/d
print(channel_fid)

<tf.Tensor: shape=(), dtype=float64, numpy=0.04181008687536769>

# Selfmade synthetic data

In [28]:
import sys
sys.path.insert(0, '../../../src_tf/')

from synthetic_data import generate_map_data
from quantum_channel import ChoiMapStatic
from optimization import Logger

In [36]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

data2 = np.load('data/data_diogo/Tgt_Map.npy', allow_pickle=True)
super_operator_ref = data2[0]

channel_target = ChoiMapStatic(super_operator_ref, mode= "super_operator")

inputs_map, targets_map = generate_map_data(channel_target, IdealSPAM(d), N_map=None, shots=None, grid=True, batch_size=1000
)

In [37]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                            rank = d**2,
                                            spam = IdealSPAM(d),
                                            ),
                        loss_function = ProbabilityMSE(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function=ProbabilityRValue(),
                                        loss_function_val = channel_fidelity_loss,
                                        sample_freq=100,)
                    )

In [40]:
model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [channel_target],
            num_iter = 2000,
            N = 500,
            verbose=True,
        )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.9999750807324187 -0.9901117626236576
0.9999768025345733 -0.9907624110479312
0.9999809031604473 -0.9913722778828955
0.9999827748085977 -0.9919391355532091
0.9999847573306689 -0.9924679801533448
0.9999869516570384 -0.9929572366654352
0.999988090642118 -0.9934145158714139
0.9999901616512736 -0.9938373331368959
0.999991162087131 -0.9942328206453238
0.9999920779707314 -0.9945998763480808
0.9999928225031672 -0.994942993656358
0.9999937361542311 -0.9952605939293202
0.9999947617706199 -0.99555711395683
0.9999953521886357 -0.995830503358683
0.9999957378298865 -0.9960852307550779
0.9999963442293132 -0.9963226667999662
0.9999966532225522 -0.9965427518425067
0.999997106966605 -0.9967483458325929
0.9999974070726244 -0.996939366456594
0.9999977152236053 -0.9971187605537524
0.9999978645944607 -0.9972838784283603


In [41]:
channel_fid = channel_fidelity(channel_target, model.channel)
print(channel_fid)

tf.Tensor(0.9972838858371896, shape=(), dtype=float64)
